<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li><li><span><a href="#Общий-вывод-по-проекту" data-toc-modified-id="Общий-вывод-по-проекту-7"><span class="toc-item-num">7&nbsp;&nbsp;</span><b>Общий вывод по проекту</b></a></span></li><li><span><a href="#Общий-вывод-по-проекту-В2" data-toc-modified-id="Общий-вывод-по-проекту-В2-8"><span class="toc-item-num">8&nbsp;&nbsp;</span><b>Общий вывод по проекту В2</b></a></span></li></ul></div>

# Отток клиентов

Цели и задачи:

   - Проанализировать исторические данные о поведении клиентов банка, расторжении договоров. Спрогнозировать уход клиента из банка.
   - Построить модель классификации, метрика качества f1 (минимальное значение 0.59)
   - Измерить AUC-ROC, сравнить её значение с f1-мерой.

План выполнения работы:

    Подготовка данных
     1.1 Загрузка данных
     1.2 Подготовка признаков
    Исследование задачи
     2.1 Решающее дерево
     2.2 Случайный лес
     2.3 Логистическая регрессия
    Борьба с дисбалансом
     3.1 Взвешивание классов
     3.2 Upsampling и Downsampling
    Проверка модели на тестовой выборке
    Общий вывод


Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [ ]:
%pip install phik

In [ ]:
#загрузим необходиме библиотеки
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import phik
from phik.report import plot_correlation_matrix
from phik import report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

In [ ]:
#загрузим файл с данными
try:
    df = pd.read_csv('/datasets/Churn.csv')
except:
    df = pd.read_csv('Churn.csv')

In [ ]:
df.head()

In [ ]:
#приведем нименования к нижнему регистру
df.columns = df.columns.str.lower()

In [ ]:
#проверим на наличие дубликатов
df.duplicated().sum()

 дубликаты отсутствуют



In [ ]:
df.info()

 В датафрейме представлены разные типы данных. Пропуски есть только в столбце 'tenure'.



In [ ]:
#удалим признаки RowNumber, CustomerId и Surname как не несущие полезной информации
df.drop(['rownumber', 'customerid', 'surname'], axis=1, inplace=True)

 Признаки rownumber, customerid - не несут полезной информации, т.к. являются техническими идентификаторами. surname содержит имена клиентов, которые никак не влияют на вероятность ухода клиента из Банка.


In [ ]:
#проверим на наличие пропусков
df.isnull().sum()

Почти 10% данных признака Tenure пропущено. Посмотри, как распределены оставшиеся.

In [ ]:
df.tenure.hist(bins=50,figsize=(8, 3))
plt.show()

Данные рапределены относительно равномерно. Заменим пропущенные значения медианой.

In [ ]:
df.tenure.fillna(df.tenure.median(), inplace=True)

In [ ]:
#построим матрицу корреляции
phik_overview = df.phik_matrix()
phik_overview.round(2)

In [ ]:
sns.heatmap(df.phik_matrix(), annot = True, fmt='.1g', cbar_kws= {'orientation': 'horizontal'}, cbar=False);

Значимых зависимостей между данными не наблюдается

In [ ]:
#преобразуем категориальные признаки в численные
df_ohe = pd.get_dummies(df, drop_first=True).rename(columns = {
    'geography_Germany':'germany',
    'geography_Spain':'spain', 
    'gender_Male':'male', 
})

df_ohe.head()

In [ ]:
df_ohe.info()

### Выводы
1. Данные загружены успешно. Дубликатов нет. Около 10% пропусков в столбце "tenure" заменены средним значением.
2. Удалены признаки 'rownumber', 'customerid', 'surname' как не несущие нужной информации.
3. Числовые признаки приведены к стандартному виду.
4. Анализ матрицы корреляции не вывел значимых зависимостей между данными.

## Исследование задачи

In [ ]:
#посмотрим на распределение целевого признака
df_ohe.exited.value_counts().plot(kind='pie', autopct='%1.1f%%', shadow=True, figsize=(5,4))
plt.legend(["Клиент ушёл", 'Клиент остался'], fontsize=8, shadow=True, facecolor='w')
plt.title('Распределение клиентов')
plt.tight_layout()
plt.axis('off');

In [ ]:
#разделим данные на обучающую, тестовую и валидационную выборку в пропорциях 3:1:1 соответственно
features = df_ohe.drop(['exited'], axis=1)
target = df_ohe.exited
train_features, valid_features, train_target, valid_target = \
                 train_test_split(features, target, \
                 test_size=0.4, stratify = target, random_state=12345)
valid_features, test_features, valid_target, test_target = \
                train_test_split(valid_features, valid_target, test_size=0.5, random_state=12345)
train_features.shape

In [ ]:
valid_features.shape

In [ ]:
test_features.shape

In [ ]:
#приведем данные к одному масштабу
numeric = ['creditscore', 'age', 'balance', 'estimatedsalary']
scaler = StandardScaler()
scaler.fit(train_features[numeric])
pd.options.mode.chained_assignment = None

train_features[numeric] = scaler.transform(train_features[numeric])

valid_features[numeric] = scaler.transform(valid_features[numeric])

test_features[numeric] = scaler.transform(test_features[numeric])

Попробуем обучить модель без учета дисбаланса классов

In [190]:
def model_fabric(model, depth, est):
    if model == LogisticRegression:
        return LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
    elif model == DecisionTreeClassifier:
        return DecisionTreeClassifier(random_state=12345, max_depth=depth)
    elif model == RandomForestClassifier:
        return RandomForestClassifier(random_state=12345, criterion='gini', n_estimators=est, max_depth=depth)
    else:
        print('Неизвестная модель')

def generator(model_name, train_features, train_target, valid_features, valid_target, depth_range=range(1, 2), est_range=range(1, 2)):
    best_result_f1 = 0
    best_result_roc_auc = 0
    best_depth_f1 = 0
    best_depth_roc_auc = 0
    best_est_f1 = 0
    best_est_roc_auc = 0

    for est in est_range:
        for depth in depth_range:
            model = model_fabric(model_name, depth, est)
            model.fit(train_features, train_target) 
            predictions = model.predict(valid_features) 
            result_f1 = f1_score(valid_target, predictions) 
            if result_f1 > best_result_f1:
                best_result_f1 = result_f1
                best_depth_f1 = depth
                best_est_f1 = est
            probabilities_valid = model.predict_proba(valid_features)
            probabilities_one_valid = probabilities_valid[:, 1]
            result_roc_auc = roc_auc_score(valid_target, probabilities_one_valid)
            if result_roc_auc > best_result_roc_auc:
                best_result_roc_auc = result_roc_auc
                best_depth_roc_auc = depth
                best_est_roc_auc = est

    return best_result_f1, best_result_roc_auc, best_depth_f1, best_depth_roc_auc, best_est_f1, best_est_roc_auc

In [191]:
best_result_f1, best_result_roc_auc, *other = generator(LogisticRegression, train_features, 
                                                        train_target, valid_features, valid_target)

print('F1-метрика:', best_result_f1, 
      '\nROC_AUC:',best_result_roc_auc)

F1-метрика: 0.3050847457627119 
ROC_AUC: 0.770301008235326


In [192]:
best_result_f1, best_result_roc_auc, best_depth_f1, best_depth_roc_auc, *other = generator(DecisionTreeClassifier, 
                                                        train_features, train_target, valid_features, 
                                                        valid_target, depth_range=range(1, 30))

print('F1-метрика:', best_result_f1, 
      '\nROC_AUC:',best_result_roc_auc)

F1-метрика: 0.5607476635514019 
ROC_AUC: 0.8323735255174299


In [193]:
%%time

best_result_f1, best_result_roc_auc, best_depth_f1, best_depth_roc_auc, best_est_f1, best_est_roc_auc = generator(RandomForestClassifier, train_features, train_target, valid_features, valid_target, depth_range=range(1, 30), est_range=range(1, 60))

print('F1-метрика:', best_result_f1, 
      '\nROC_AUC:',best_result_roc_auc)

F1-метрика: 0.5907046476761619 
ROC_AUC: 0.8602060977334973
CPU times: user 3min 1s, sys: 724 ms, total: 3min 1s
Wall time: 3min 1s


### Выводы
1. Целевой признак распределен в отношении 4:1.
2. При обучении обучении моделей без учета дисбаланса лучший показатель F1-точности у модели Случайного леса - 0.59. Качество модели удовлетворяет условию технического задания, но попробуем ее улучшить.
3. Лучшее значение метрики AUC_ROC - 0,84 также дало обучение на модели случайного леса.

## Борьба с дисбалансом

Для нивелирования дибаланса применим 2 метода - upsampling и downsampling

In [194]:
#увеличим количество объектов редкого класса
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [195]:
features_upsampled, target_upsampled = upsample(train_features, train_target, 4)
target_upsampled.value_counts(normalize=True)

1    0.50569
0    0.49431
Name: exited, dtype: float64

In [196]:
#уменьшим количество объектов многочисленного класса
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [197]:
features_downsampled, target_downsampled = downsample(train_features, train_target, fraction=0.25)

In [198]:
target_downsampled.value_counts(normalize=True)

1    0.505795
0    0.494205
Name: exited, dtype: float64

Обучим модели на новых данных

In [199]:
%%time

models_and_params = [
    [ DecisionTreeClassifier(), { 'max_depth': range (1,31) } ],
    [ LogisticRegression(), {'C': np.logspace(-4, 4, 50)} ],
    [ RandomForestClassifier(), {'n_estimators': range (10, 91, 10), 'max_depth': range (1,20)} ]
]
scorings = ['f1', 'roc_auc']
directions = ['down']

def fitter(model, params, scoring, direction):
    method = GridSearchCV(model, params, cv=5, n_jobs=-1, scoring=scoring)
    method.fit(features_downsampled, target_downsampled)

    return [model, scoring, direction, method.best_score_, method.best_params_]

data = []
for model, params in models_and_params:
    for scoring in scorings:
        for direction in directions:
            data.append( fitter(model, params, scoring, direction) )

CPU times: user 2.93 s, sys: 810 ms, total: 3.74 s
Wall time: 19.8 s


In [200]:
best_result_f1, best_result_roc_auc, *other = generator(LogisticRegression, 
                                                        features_upsampled, target_upsampled, valid_features, 
                                                        valid_target)

data.append(['LogisticRegression()', 'f1', 'up', best_result_f1, ''])
data.append(['LogisticRegression()', 'roc_auc', 'up', best_result_roc_auc, ''])


In [205]:
%time
best_result_f1, best_result_roc_auc, *other = generator(RandomForestClassifier, features_upsampled, target_upsampled, valid_features, valid_target, depth_range=range(1, 30), est_range=range(1, 60))

data.append(['RandomForestClassifier()', 'f1', 'up', best_result_f1, ''])
data.append(['RandomForestClassifier()', 'roc_auc', 'up', best_result_roc_auc, ''])
 

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 10 µs


In [206]:
best_result_f1, best_result_roc_auc, best_depth_f1, best_depth_roc_auc, *other = generator(DecisionTreeClassifier, 
                                                        features_upsampled, target_upsampled, valid_features, 
                                                        valid_target, depth_range=range(1, 30))
data.append(['DecisionTreeClassifier()', 'f1', 'up', best_result_f1, ''])
data.append(['DecisionTreeClassifier()', 'roc_auc', 'up', best_result_roc_auc, ''])

In [207]:
df_models = pd.DataFrame(data, columns =['model', 'scoring', 'direction', 'best_score', 'params'])

#отсортируем данные по значениям точности
df_models.sort_values(by='best_score', ascending=False)

,model,scoring,direction,best_score,params
5,RandomForestClassifier(),roc_auc,down,0.858756,"{'max_depth': 7, 'n_estimators': 80}"
9,RandomForestClassifier(),roc_auc,up,0.856836,
11,DecisionTreeClassifier(),roc_auc,up,0.831573,
1,DecisionTreeClassifier(),roc_auc,down,0.823880,{'max_depth': 5}
4,RandomForestClassifier(),f1,down,0.779339,"{'max_depth': 8, 'n_estimators': 90}"
7,LogisticRegression(),roc_auc,up,0.775381,
3,LogisticRegression(),roc_auc,down,0.763954,{'C': 0.040949150623804234}
0,DecisionTreeClassifier(),f1,down,0.752474,{'max_depth': 6}
2,LogisticRegression(),f1,down,0.711630,{'C': 0.0001}
8,RandomForestClassifier(),f1,up,0.615205,


### Выводы
1. Upsampling положительного класса и downsampling отрицательного класса дали необходимый прирост точности моделей.
2. Лучшие показатели точности на тренировочной выборке у модели Случайного леса.

## Тестирование модели

In [208]:
model = RandomForestClassifier()
model.fit(features_downsampled, target_downsampled)
predicted_test = model.predict(test_features)
f1 = f1_score(test_target, predicted_test)
probabilities_test = model.predict_proba(test_features)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score (test_target, probabilities_one_test)
print(' f1 = {}, \n roc_auc = {}'.format(round(f1,2), round(auc_roc,2)))

 f1 = 0.62, 
 roc_auc = 0.86


## Выводы


1. Был проведен первичный анализ данных, выявлен явный дисбаланс классов, отрицательного к положительному - 4 к 1.
2. Произведена предобработка данных, заполннение пропуски средним значением, прошкалированы количественные переменные, сделан One_hot_encoding.
3. Модель Случайного леса до борьбы с дисбалансом показывала результат метрики f1 - 59%.
4. Произвели downsampling положительного класса, благодаря чему удалось повысить f1 меру до 62%.
    

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*